# SQL nâng cao

## SQL động với Exec

SQL động là nhóm sử dụng nâng cao của SQL, cho phép người dùng tự động hóa các script. Sử dụng SQL động cho phép tùy biến các câu lệnh khi truy vấn và xử lý dữ liệu. Khi sử dụng SQL động, ta cần dùng `EXEC`


```sql
-- Cách 1
declare @id int
set @id = '4'
exec('select * from ACCOUNT where ACCOUNT_ID = ' + @id)

-- Cách 2
declare @id varchar(100)
set @id = '4'
declare @sql varchar(1000)
set @sql = 'select * from ACCOUNT where account_id = ' + @id
print @sql -- Đảm bảo câu lệnh thực hiện đúng
exec(@sql)

```

**Lưu ý khi sử dụng SQL động**

- Lưu ý dấu cách 


```sql
EXEC(' SELECT col1, col2, col3 ' +
     ' FROM ' + @tblname +
     ' WHERE keycol = ''' + @key + '''')

```

- Luôn dùng print để đảm bảo viết đúng câu lệnh trước khi sử dụng exec



```sql
declare @month varchar(8) = (select max(date) from ##A)
print('select * from ##B where month = ' + @month)

```

## Thủ tục & hàm

### Procedure

**Hiển thị tất cả các procedure đang lưu trong hệ thống**

```sql
-- Tất cả các procedure
SELECT name, 
       type
FROM dbo.sysobjects
WHERE (type = 'P')
 
-- Các procedure do người dùng tạo 
  select * 
  from master.information_schema.routines 
  where routine_type = 'PROCEDURE' 
        and Left(Routine_Name, 3) NOT IN ('sp_', 'xp_', 'ms_')

```

- **Xóa procedure**



```sql
drop procedure cast_data


```

### Tạo procedure

- Procedure không có biến


```sql
-- Tạo procedure
create procedure getID_normal 
as
select * from ACCOUNT
-- Chạy procedure
execute getID_normal

```

- Procedure nhiều biến



```sql
--Tạo procedure
create procedure PROD_CD 
  @prod nvarchar(30) = NULL,
  @AVAIL_BALANCE float
as
  select * from ACCOUNT
  where PRODUCT_CD = @prod and
  AVAIL_BALANCE >= @AVAIL_BALANCE
GO

--Chạy procedure
PROD_CD @prod = 'CHK', @AVAIL_BALANCE = 10000

```

**Lưu ý**: Trong SQL có 2 loại biến: 

- User defined variable: Bắt đầu bằng dấu **@**
- System variable: Bắt đầu bằng dấu **@@** - loại này chỉ xuất hiện sau khi đã thực hiện xong 1 số câu lệnh


```sql
select * from [DimCustomer]
select @@ROWCOUNT

```

---

**Lưu ý**: Gán kết quả 1 procedure vào biến



```sql
drop procedure if exists return_date
create procedure [dbo].[return_date] (@no_date int = 0, @no_date2 int = 2,  @result varchar(8) output)
as
begin 
set @result = (select convert(varchar(8), getdate() - @no_date + @no_date2, 112))
end

declare @result varchar(8)
execute return_date 5, 1, @result output
print @result

```

### Hàm (function)



```sql
DROP FUNCTION IF EXISTS fn_return_date

CREATE FUNCTION [dbo].[fn_return_date] (@no_date INT = 0)
RETURNS INT
AS
BEGIN
	RETURN (
			SELECT cast(convert(VARCHAR(8), getdate() - @no_date, 112) AS INT)
			)
END;

-- Cách 1
declare @result int
select @result = [dbo].[fn_return_date](4)
print @result

-- Cách 2
declare @result int
set @result = ([dbo].[fn_return_date](4))
print @result

```

## Câu điều kiện If...Else


```sql
if 1 = 1
begin
print 'wrong'
print 'not sure'
end
else
begin
print 'right'
print 'absolutely right'

```

**Lưu ý**: Sau mệnh đề else, if, nếu có nhiều câu lệnh cùng thực hiện, cần khai báo begin...end

---

Lưu ý: Ta có thể sử dụng câu điều kiện `if...else` để kiểm tra  điều kiện để thực thi tiếp

- Kiểm tra điều kiện tồn tại của bảng
- Nếu tồn tại, chạy tiếp
- Nếu không tồn tại, skip các câu lệnh tiếp theo

```sql
--SQL CMD
:on error exit

declare @tbl sysname
set @tbl = 'TBL_I2B_USERS_F0_20180502'

IF NOT EXISTS (SELECT * FROM ANHNT67.DBS_DAILY.DBO.SYSOBJECTS 
				WHERE NAME = @tbl)
BEGIN
raiserror('Table does not exist', 15, 10)
print(@tbl)
return
END

EXEC('select top 10 * FROM ANHNT67.DBS_DAILY.DBO.' + @tbl)

```

## Vòng lặp với bảng

SQL chỉ chạy được vòng lặp `WHILE`


```sql
DROP FUNCTION IF EXISTS fn_return_date

CREATE FUNCTION [dbo].[fn_return_date] (@no_date INT = 0)
RETURNS INT
AS
BEGIN
	RETURN (
			SELECT cast(convert(VARCHAR(8), getdate() - @no_date, 112) AS INT)
			)
END;

declare @result int
select @result = [dbo].[fn_return_date](4)
print @result

drop table if exists #temp
create table #temp (date int)
select * from #temp

insert into #temp (date)
select [dbo].[fn_return_date](1) as date
insert into #temp (date)
select [dbo].[fn_return_date](2) as date
insert into #temp (date)
select [dbo].[fn_return_date](3) as date

drop table if exists #temp2
select * 
into #temp2
from #temp

alter table #temp2
add new varchar(20)

select * from #temp2

update #temp2 
set new = cast(date as varchar(20)) + 'abc'


--Reference
drop table if exists #ref

create table #ref (
[index] int identity(1,1) not null,
[date] int
)

insert into #ref (date)
select [date] from #temp order by [date]

select * from #ref

-- VERSION 1: DOES NOT WORK

declare @loopcounter int, @maxcounter int 
set @loopcounter = (select min(date) from #temp2)
set @maxcounter = (select max(date) from #temp2)

while (@loopcounter <= @maxcounter)
begin
	select * from #temp2 where
	date = @loopcounter
	set @loopcounter = @loopcounter + 1
end


-- VERSION 2: WORK WELL
drop table if exists #result
create table #result (date int, new varchar(20))
select * from #result

declare @loopcounter int, @maxcounter int 
set @loopcounter = (select min([index]) from #ref)
set @maxcounter = (select max([index]) from #ref)

while (@loopcounter <= @maxcounter)
begin
	declare @date int 
	set @date = (select [date] from #ref where [index] = @loopcounter)
	
	insert into #result ([date], new)
	select * from #temp2 where
	[date] = @date
	set @loopcounter = @loopcounter + 1
end

select * from #result

```

## Đặt job trong SQL

Khi làm việc với SQL, một trong những yêu cầu rất quan trọng là phải tự động hóa code SQL theo thơi gian. Yêu cầu này có thể được xử lý bằng cách đặt job thông qua các bước lớn sau.

- Active email tự động
- Đăng ký email
- Active SQL Server Agent
- Đặt job

### Bước 1: Active email tự động

- Thực hiện câu lệnh  sau trên SQL để active gửi Database Mail. Do khi cài SQL, mặc định Database mail chưa được active nên khi sử dụng ta cần active database này

```sql
sp_configure 'show advanced options', 1;
GO
RECONFIGURE;
GO
sp_configure 'Database Mail XPs', 1;
GO
RECONFIGURE
GO

```

### Bước 2: Đăng kí email trong Database mail

- Tại màn hình SQL >> Object Explore >> Management >> Database Mail >> Tick vào Set up database Mail by performing the following tasks >> Next >> Profile name điền tên id của mình muốn >> Add >> Điền đẩy đủ thông tin như hình bên dưới với thông tin email >> Next
  
- Mục đích của bước này là sau khi hoàn thành Job ta sẽ có 1 email thông báo Job đã thành công hoặc gửi thẳng cho client là dữ liệu đã sẵn sàng để có thể sử dụng
  


```sql
ID: email
Pass: pwd

```

  
![Template](Images/job_1.png) 

---

![Template](Images/job_2.png)

---

### Bước 3: Active SQL Server Agent

- Tại `Object Explore` >> Click chuột phải vào SQL Server Agent >> Start

### Bước 4: Đặt Job

- Chuột phải vào SQL Server Agent >> New >> Điền Name vào mục General >> Qua mục Steps >> Tạo các bước của Job tại New >> Điền Step Name >> Paste code automate vào phần Command. Ví dụ có thể sử dụng tại code bên dưới 

![Template](Images/job_3.png)

---

![Template](Images/job_4.png)

---

```sql
IF EXISTS (SELECT * FROM [BA].[INFORMATION_SCHEMA].[TABLES] WHERE [TABLE_NAME] = 'CREDIT_CARD_TRANSACTION_SUCCESSFUL_MASTER')
DROP TABLE [BA].[dbo].[CREDIT_CARD_TRANSACTION_SUCCESSFUL_MASTER]

EXEC('
SELECT  ID,
		YEARMONTH,
		TARGET_ID,
		TRANS_DATE,
		TRANS_CURR,
		TRANS_AMOUNT_QD,
		TRANS_DETAILS,
		MERCHANT_ID,
		TRANS_CITY,
		TRANS_COUNTRY,
		BI_TRANS_TYPE,
		SIC_CODE,
		CIF,
		PRODUCT_DETAIL
INTO [BA].[dbo].[CREDIT_CARD_TRANSACTION_SUCCESSFUL_MASTER]
FROM [BA].[dbo].[CREDIT_CARD_TRANSACTION_SUCCESSFUL_2018]
UNION ALL
SELECT  ID,
		YEARMONTH,
		TARGET_ID,
		TRANS_DATE,
		TRANS_CURR,
		TRANS_AMOUNT_QD,
		TRANS_DETAILS,
		MERCHANT_ID,
		TRANS_CITY,
		TRANS_COUNTRY,
		BI_TRANS_TYPE,
		SIC_CODE,
		CIF,
		PRODUCT_DETAIL
FROM [BA].[dbo].[CREDIT_CARD_TRANSACTION_SUCCESSFUL_2017]
')

EXEC('
ALTER TABLE [BA].[dbo].[CREDIT_CARD_TRANSACTION_SUCCESSFUL_MASTER]
ADD SIC_ID VARCHAR(MAX), MCC_CATEGORY VARCHAR(MAX), MCC_CATEGORY_AIA VARCHAR(MAX)
')

EXEC('
UPDATE [BA].[dbo].[CREDIT_CARD_TRANSACTION_SUCCESSFUL_MASTER]
SET SIC_ID = A.SIC_CODE collate database_default,
	MCC_CATEGORY = A.CATEGORIES
FROM [BA].[dbo].[MCC_CATEGORY_20170808] A
WHERE [BA].[dbo].[CREDIT_CARD_TRANSACTION_SUCCESSFUL_MASTER].SIC_CODE = A.SIC_ID		

UPDATE [BA].[dbo].[CREDIT_CARD_TRANSACTION_SUCCESSFUL_MASTER]
SET MCC_CATEGORY_AIA = A.MCC_CATEGORY
FROM [BA].[dbo].[MCC_CATEGORY_AIA_20180307] A
WHERE [BA].[dbo].[CREDIT_CARD_TRANSACTION_SUCCESSFUL_MASTER].SIC_CODE = A.SIC_ID
')

```

  
  - Tạo bước gửi email đến Client >> Đặt tên là SUCCESSFUL JOB >> ta làm tương tự các bước như trên với code trong mục command như sau. 
  


```sql
USE [msdb]
DECLARE @REPORT_DATE DATE = (select MAX(TRANS_DATE) from  BA..CREDIT_CARD_TRANSACTION_SUCCESSFUL_MASTER);
DECLARE @SUBJECT VARCHAR(250) = '[JOB DONE] - CREDIT CARD TRANSACTION - '+convert(varchar(8),@REPORT_DATE,112);
EXEC
sp_send_dbmail
@profile_name ='thanh',
@recipients ='thanhnm3@vpbank.com.vn; truongnh3@vpbank.com.vn; lambt1@vpbank.com.vn',
@subject = @SUBJECT,
@body =N'CREDIT CARD TRANSACTION IS AVAILABLE',
@execute_query_database ='msdb'

```

  
- Tạo bước gửi email đến chính mình khi JOB bị failed >> Đặt tên là FAILED JOB >> ta làm tương tự các bước như trên với code trong mục command như sau.
  


```sql
USE [msdb]
DECLARE @REPORT_DATE DATE = (select MAX(TRANS_DATE) from  BA..CREDIT_CARD_TRANSACTION_SUCCESSFUL_MASTER);
DECLARE @SUBJECT VARCHAR(250) = '[JOB DONE] - CREDIT CARD TRANSACTION - '+convert(varchar(8),@REPORT_DATE,112);
EXEC
sp_send_dbmail
@profile_name ='thanh',
@recipients ='thanhnm3@vpbank.com.vn',
@subject = @SUBJECT,
@body =N'CREDIT CARD TRANSACTION HAS BEEN FAILED',
@execute_query_database ='msdb'

```

  
- Sửa lại cách thức chạy code của các bước >> Double Click vào step 1 >> Chọn Advanced >> Tại On failure action >> Chọn go to FAILED JOB (step đặt mail báo failed)

![Template](Images/job_5.png)

**Lưu ý**

- Code SQL được đưa vào trong JOB cần là code động 

## Pivot & Unpivot

```sql
-- Tạo bảng
create table #student
(
id int,
mark1 float,
mark2 float,
mark3 float
)

insert into #student values (1, 1,2,3)
insert into #student values (2, 5,6,7)
insert into #student values (3, 7,4,2)

-- UNPIVOT

SELECT * FROM
(SELECT *
FROM #Student) a
UNPIVOT
(value_var FOR group_var IN (Mark1, Mark2, Mark3)
) b

-- PIVOT

SELECT * 
into #student_pivot FROM
(SELECT *
FROM #Student) a
UNPIVOT
(value_var FOR group_var IN (Mark1, Mark2, Mark3)
) b


select * from 
(select * from #student_pivot) a
pivot 
(sum(value_var) for group_var in (mark1, mark2, mark3)) b

```

### Procedure pivot

- Dynamic sql


```sql
declare @data nvarchar(30)
declare @group nvarchar(30)
set @data = '#student_pivot'
set @group = STUFF(
(SELECT ',' + b.group_var
              FROM 
			  (select distinct group_var from #student_pivot
			  where group_var not like 'mark1') b
              FOR XML PATH ('')),1,1,'')

declare @sqlstr nvarchar(max)
set @sqlstr = N'select * from 
(select * from '+ @data +') a
pivot 
(sum(value_var) for group_var in (' + @group + ')) b
'
exec sp_executesql @sqlstr

```

- Procedure



```sql
drop procedure if exists sql_pivot
create procedure sql_pivot  @data nvarchar(max),
							@group nvarchar(max), -- Variable to pivot
							@row nvarchar(max), -- Column to keep as key
							@value nvarchar(max), -- Column to calculate
							@function nvarchar(max) = 'sum', -- Function to calculate
							@stored_data nvarchar(max) = '##pivot_result' -- Store data result
as
begin
declare @sql nvarchar(max)
set @sql = '
declare @group_value nvarchar(max)
set @group_value = STUFF(
(SELECT '','' + b.'+@group+'
              FROM 
			  (select distinct '+ @group +' from '+ @data +') b
              FOR XML PATH ('''')),1,1,'''')

declare @sqlstr nvarchar(max)
set @sqlstr = N''
drop table if exists '+ @stored_data +'
select * 
into '+ @stored_data +'
from 
(select '+ @group+ ',' + @row + ', '+@value +'  from '+ @data +') a
pivot 
('+@function+'('+ @value +') for '+@group+' in ('' + @group_value + '')) b
''
exec sp_executesql @sqlstr
'
exec sp_executesql @sql
end;

```


- Ví dụ:


```sql
sql_pivot @data = '##student_pivot', @row = 'id',
@group= 'group_var', @value = 'value_var', @function = 'sum'
select * from ##pivot_result

```

### Procedure unpivot

- Dynamic sql: Cho phép nhiều ID

- Cách 1: Chỉ sử dụng với các data trong máy local


```sql
declare @data nvarchar(max) = 'student'
declare @id_var nvarchar(max) = '''id'',''mark1'''
declare @group_var nvarchar(max) = 'group_var'
declare @value_var nvarchar(max) = 'value_var'

declare @sql nvarchar(max)
set @sql = '
declare @cols AS NVARCHAR(MAX)
   select @cols = STUFF((SELECT '','' + QUOTENAME(column_name) 
                    FROM (
                        select   table_name, column_name, ordinal_position, data_type
                        from   information_schema.columns
                        where   table_name = '''+@data+''' and 
						column_name not in ('+@id_var+') 
                        ) cols
                    ORDER BY ordinal_position               
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
select @cols

declare @key AS NVARCHAR(MAX)
   select @key = STUFF((SELECT '','' + QUOTENAME(column_name) 
                    FROM (
                        select   table_name, column_name, ordinal_position, data_type
                        from   information_schema.columns
                        where   table_name = '''+@data+''' and 
						column_name in ('+@id_var+') 
                        ) cols
                    ORDER BY ordinal_position               
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
select @key

declare @sqlstr nvarchar(max)
set @sqlstr = N''
            select ''+@key+'', '+@group_var+', '+@value_var+'
            from(
               select *
               from '+@data+'
            ) as a
            unpivot
            (
              '+@value_var+' for '+@group_var+' in ('' + @cols + '')
            ) as b''
print @sqlstr
exec sp_executesql @sqlstr
'
  
exec sp_executesql @sql

```

- Cách 2: Khái quát dynamic sql



```sql
declare @db nvarchar(max) = 'tempdb'
declare @data nvarchar(max) = '##student'
declare @id_var nvarchar(max) = '''id'''
declare @group_var nvarchar(max) = 'group_var'
declare @value_var nvarchar(max) = 'value_var'
declare @stored_data nvarchar(max) = '##a'
declare @sql nvarchar(max) 

set @sql = N'
drop table if exists ##all_cols
select name into ##all_cols
from '+@db+'.sys.columns 
where object_id in (select object_id from '+@db+'.sys.tables where name like ''%'+@data+''')

declare @cols AS NVARCHAR(MAX)
   select @cols = STUFF((SELECT '','' + QUOTENAME(name) 
                    FROM (
                        select name from ##all_cols 
                        where name not in ('+@id_var+') 
                        ) cols            
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
select @cols

declare @key AS NVARCHAR(MAX)
   select @key = STUFF((SELECT '','' + QUOTENAME(name) 
                    FROM (
                        select name from ##all_cols 
                        where name in ('+@id_var+') 
                        ) cols            
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
select @key

--Run procedure

declare @sqlstr nvarchar(max)
set @sqlstr = N''
			drop table if exists '+@stored_data+'
            select ''+@key+'', '+@group_var+', '+@value_var+'
			into '+@stored_data+'
            from(
               select *
               from '+@db+'.dbo.['+@data+']
            ) as a
            unpivot
            (
              '+@value_var+' for '+@group_var+' in ('' + @cols + '')
            ) as b
			select * from '+@stored_data+'
			''
print @sqlstr
exec sp_executesql @sqlstr
'
exec sp_executesql @sql

```

**Lưu ý**: Các bảng phải là bảng trong db hoặc bảng lưu ở global environment

- Procedure


```sql
drop procedure if exists sql_unpivot
create procedure sql_unpivot 
 @db nvarchar(max) = 'tempdb', --Database contains data
@data nvarchar(max), -- Name of data
@id_var nvarchar(max), -- Key variables
@group_var nvarchar(max) = 'group_var', --Name of groups variable
@value_var nvarchar(max) = 'value_var', -- Name of value variable
@stored_data nvarchar(max) = '##unpivot_result'  -- Data storage
as 
begin
declare @sql nvarchar(max) 
set @sql = N'
drop table if exists ##all_cols
select name into ##all_cols
from '+@db+'.sys.columns 
where object_id in (select object_id from '+@db+'.sys.tables where name like ''%'+@data+''')

declare @cols AS NVARCHAR(MAX)
   select @cols = STUFF((SELECT '','' + QUOTENAME(name) 
                    FROM (
                        select name from ##all_cols 
                        where name not in ('+@id_var+') 
                        ) cols            
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
-- Show columns
print @cols 

declare @key AS NVARCHAR(MAX)
   select @key = STUFF((SELECT '','' + QUOTENAME(name) 
                    FROM (
                        select name from ##all_cols 
                        where name in ('+@id_var+') 
                        ) cols            
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
-- Show key
print @key 

--Run procedure

declare @sqlstr nvarchar(max)
set @sqlstr = N''
			drop table if exists '+@stored_data+'
            select ''+@key+'', '+@group_var+', '+@value_var+'
			into '+@stored_data+'
            from(
               select *
               from '+@db+'.dbo.['+@data+']
            ) as a
            unpivot
            (
              '+@value_var+' for '+@group_var+' in ('' + @cols + '')
            ) as b
			select * from '+@stored_data+'
			''
print @sqlstr
exec sp_executesql @sqlstr
'
exec sp_executesql @sql
end

```

```sql
exec sql_unpivot @db = 'loan_db', @data = 'MONTHLY_LCY_AMOUNT - M74 - BICDATA', 
@id_var = '''id'', ''YEARMONTH'', ''APP'',''ACCTNO'', ''CIF'', ''AVERAGE_BAL'', ''BI_SEGMENT'''

```

**Lưu ý**:

- Bắt buộc phải có alias sau bảng


## Chia một cột thành nhiều cột


```sql
CREATE TABLE tbl_SplitColumns (col1 varchar(max))
GO
 
INSERT INTO tbl_SplitColumns 
VALUES ('part1 part2 part3'),('abc xyz pqr'),('one two three'), ('Hello man nua nha')
GO


select * from tbl_SplitColumns

DECLARE @delimiter VARCHAR(5)
SET @delimiter=' '  
;WITH CTE AS
( 
    SELECT
        col1,
        CAST('<A>' + REPLACE(col1, @delimiter , '</A><A>') + '</A>' AS XML) AS XMLString
    FROM tbl_SplitColumns 
)
select * from CTE
SELECT
     XMLString.value('/A[1]', 'varchar(10)') As C1
     ,XMLString.value('/A[2]', 'varchar(10)') As C2
     ,XMLString.value('/A[3]', 'varchar(10)') As C3
	 ,XMLString.value('/A[4]', 'varchar(10)') As C4
  FROM CTE 
GO

```